In [7]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr

In [18]:
path =  '/projects/0/FWC2/Spatial_dependence/Paper2'
geounit = 107
rps = [2,5,10,25,50,100,250,500,1000]
i = int(sys.argv[1])
rp = rps[i]

In [25]:
# Inuput file locations
src_station_loc = os.path.join(path,'Hazard/Source_station/src_station_rp1000_globe.nc')
dmg_file_loc = os.path.join(path, 'Impact/GLOFRIS')
geo_loc = os.path.join(path,'Geogunit/geogunit_107_all.nc')

# Read file
src_rp1000_file = xr.load_dataset(src_station_loc)
src_rp1000 = src_rp1000_file.src.values
geo_globe = xr.load_dataset(geo_loc)
geo = geo_globe.Geogunits.values
ds_geo_station = pd.read_pickle(os.path.join(path,'Station_geogunit107/geogunit_station_globe.pkl'))
ds_station_geo = pd.read_pickle(os.path.join(path,'Station_geogunit107/station_geoguint_globe.pkl'))

In [27]:
sts = ds_station_geo.iloc[:,0]
df_dmg = pd.DataFrame(index=range(len(sts)),columns = ['Station','Dmg_perc'])
df_dmg['Station'] = sts
src = src_rp1000.copy()

# read damages
dmg_globe = xr.load_dataset(os.path.join(dmg_file_loc,'inuncoast_historical_nosub_hist_rp{:04d}_0_Built-up_Area.nc').format(rp))
dmg = dmg_globe.Impact_Results.values # clip to the basin

# Damage percentage file
perc = []

for i in range(len(sts)):
    
    st = sts[i]
    
    # calculate the total damage of this station
    dmg_total = dmg[np.where(src==st)].sum()
    
    if dmg_total == 0:
        perc.append([0.0])
        continue
    
    # get all units affected by this station
    units = ds_station_geo.iloc[i,1]
    
    dmg_perc = []
    
    for unit in units:
        # Get the points where both files show positive values
        r_geo, c_geo = np.where(geo==unit)
        geo_points = np.vstack((r_geo, c_geo)).T
        r_src, c_src = np.where(src==st)
        src_points = np.vstack((r_src, c_src)).T
    
        # Find the geo unit points that are intersected with the station-location points
        set_y = set(map(tuple, geo_points))
        idx = [tuple(point) in set_y for point in src_points]
        com_points = src_points[idx]
        r_com, c_com = com_points[:,0], com_points[:,1]
        
        dmg_unit = dmg[r_com, c_com].sum()
        dmg_perc.append(dmg_unit/dmg_total)
    
    perc.append(dmg_perc)
    
df_dmg.iloc[:,1] = perc
df_dmg.to_pickle(os.path.join(path,'Station_geogunit107/dmg_perc_rp{:04d}_globe.pkl'.format(rp))

0
14
15
16
18
20
21
22
23
24
25
26
27
28
29
30
31
41
42
44
45
46
48
49
50
51
52
53
55
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85


KeyboardInterrupt: 

In [28]:
rps[8]

1000